Imports:

In [3]:
# !pip install pandas
# !pip install numpy
# !pip install matplotlib
# !pip install scikit-learn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model as lm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier


# Data Retrieval 

In [181]:
dialogue = pd.read_csv(("https://raw.githubusercontent.com/"
 "rfordatascience/tidytuesday/master/data/"
 "2020/2020-09-08/friends.csv"))

print("Example Dialogue:")
dialogue.sample(4)


Example Dialogue:


,text,speaker,season,episode,scene,utterance
3868,(Chandler is completely astounded.),Scene Directions,1,15,8,29
33673,What-what Richard thing?,Chandler Bing,5,23,3,17
43681,Did you two...,Monica Geller,7,9,2,20
32493,Really?!,Caitlin,5,19,4,11


In [146]:
episodes = pd.read_csv(("https://raw.githubusercontent.com/"
 "rfordatascience/tidytuesday/master/data/"
 "2020/2020-09-08/friends_info.csv"))

print("Example Episode:")
episodes.sample(4)

Example Episode:


,season,episode,title,directed_by,written_by,air_date,us_views_millions,imdb_rating
94,4,22,The One with the Worst Best Man Ever,Peter Bonerz,Story by : Seth KurlandTeleplay by : Gregory S...,1998-04-30,23.20,8.5
184,8,15,The One with the Birthing Video,Kevin S. Bright,Dana Klein Borkow,2002-02-07,28.64,8.4
141,6,21,The One Where Ross Meets Elizabeth's Dad,Michael Lembeck,Story by : David J. LaganaTeleplay by : Scott ...,2000-04-27,20.60,8.6
21,1,22,The One with the Ick Factor,Robby Benson,Alexa Junge,1995-05-04,29.90,8.3


# Preprocessing

In [171]:


# 6 Main Character Names
main_character_names = ["Ross Geller", "Monica Geller",
                        "Phoebe Buffay", "Chandler Bing",
                        "Joey Tribbiani", "Rachel Green"]


#Filters Out Non Dialogue Rows
all_dialogue = dialogue[~dialogue["speaker"].isin(["Scene Directions", "#ALL#", "NA"])]


#Indexs of  MAIN CHARACTER DIALOGUE
dialogue_mc_indexs = all_dialogue["speaker"].isin(main_character_names)

#Indexs of OTHER CHARACTER DIALOGUE
dialogue_oc_indexs = ~all_dialogue["speaker"].isin(main_character_names)

#Indexs of EXCLAMATORY DIALOGUE
dialogue_exclamatory_indexs = all_dialogue["text"].str.contains(r'[A-Z]{2,}') | all_dialogue["text"].str.contains(r'!')

In [172]:
#FUNCTION TO APPLY

def sum_size(x):
    return x.str.len().sum()
                      

# Processing 1/4: Adding Episode Level Dialogue Info


In [192]:

#Table To Store Episode Level Dialogue Summary
jt_1 = all_dialogue.groupby(["season", "episode"]).agg("first").reset_index().drop(columns=["scene", "utterance", "speaker", "text"])

jt_1.head(2)

,season,episode
0,1,1
1,1,2


In [193]:
#Grouped Data
gp = all_dialogue.groupby(["season", "episode"])

#Adding Total Words In Episode
jt_1 = pd.merge(jt_1, gp["text"].size().reset_index(name="total_words"))

#Adding Total Lines In Episode
jt_1 = pd.merge(jt_1, gp["text"].agg(sum_size).reset_index(name="total_lines"))

#Adding Total Scenes In Episode
jt_1 = pd.merge(jt_1, gp["scene"].nunique().reset_index(name="total_scenes"))

#Adding Total Speakers In Episode
jt_1 = pd.merge(jt_1, gp["speaker"].nunique().reset_index(name="total_speakers"))

jt_1.head(5)

,season,episode,total_words,total_lines,total_scenes,total_speakers
0,1,1,292,16876,15,11
1,1,2,241,12249,11,14
2,1,3,237,12070,14,10
3,1,4,248,12532,16,14
4,1,5,238,12626,16,10


In [194]:

#Grouped Data (All NON-MAIN Characters Lines By Episodes)
gp = all_dialogue[dialogue_oc_indexs].groupby(["season", "episode"])


# Adding Total Lines By NON-MAIN characters
jt_1 = pd.merge(jt_1, gp["text"].size().reset_index(name="total_oc_lines"))

#Adding Total Scenes By NON-MAIN characters
jt_1 = pd.merge(jt_1, gp["scene"].nunique().reset_index(name="total_oc_scenes"))


#Grouped Data (All NON-MAIN Characters Exclamatory Lines By Episode)
gp = all_dialogue[dialogue_oc_indexs & dialogue_exclamatory_indexs].groupby(["season", "episode"])

#Adding Total Exclamatory Lines By Non Main Characters
jt_1 = pd.merge(jt_1, gp["text"].size().reset_index(name="total_oc_exclam_lines"))

jt_1.head(5)

,season,episode,total_words,total_lines,total_scenes,total_speakers,total_oc_lines,total_oc_scenes,total_oc_exclam_lines
0,1,1,292,16876,15,11,27,9,5
1,1,2,241,12249,11,14,69,8,19
2,1,3,237,12070,14,10,24,6,3
3,1,4,248,12532,16,14,31,6,10
4,1,5,238,12626,16,10,34,10,2


# Processing 2/4: Adding Character Level Dialogue Info

In [202]:
#Table To Store Character Level Dialogue Summary
# (WILL BE PIVOTED INTO ABOVE FORM BELOW)

jt_2 = all_dialogue[dialogue_mc_indexs].groupby(["season", "episode", "speaker"]).agg("first").reset_index().drop(columns=["scene", "utterance", "text"])

jt_2.head(6)

,season,episode,speaker
0,1,1,Chandler Bing
1,1,1,Joey Tribbiani
2,1,1,Monica Geller
3,1,1,Phoebe Buffay
4,1,1,Rachel Green
5,1,1,Ross Geller


In [203]:
#Grouped Data (All Main Character Dialogue By Episode, Speaker)
gp = all_dialogue[dialogue_mc_indexs].groupby(["season", "episode", "speaker"])

#Adding Total Lines Per Main Character
jt_2 = pd.merge(jt_2, gp["text"].size().reset_index(name="total_mc_line"))

#Adding Total Scenes Per Main Character 
jt_2 = pd.merge(jt_2, gp["scene"].nunique().reset_index(name="total_mc_scene"))



#Grouped Data (All Main Character Exclamatory Lines, By Episode, Speaker)
gp = all_dialogue[dialogue_mc_indexs & dialogue_exclamatory_indexs].groupby(["season", "episode", "speaker"])

#Adding Total Exclamatory Lines By Main Character
jt_2 = pd.merge(jt_2, gp["text"].size().reset_index(name="total_mc_exclam_line"))


jt_2.head(6)

,season,episode,speaker,total_mc_line,total_mc_scene,total_mc_exclam_line
0,1,1,Chandler Bing,39,9,8
1,1,1,Joey Tribbiani,39,8,11
2,1,1,Monica Geller,73,9,20
3,1,1,Phoebe Buffay,19,6,5
4,1,1,Rachel Green,48,8,15
5,1,1,Ross Geller,47,8,8


In [204]:

#Pivoting Table (To Ensure One Row Per Episode)
jt_2 = jt_2.pivot(index=["season", "episode"], columns="speaker", values=["total_mc_line", "total_mc_scene", "total_mc_exclam_line"]).reset_index(inplace=False)

#Renaming Columns + Reseting Index
jt_2.columns = ['_'.join(map(str, col)).strip() for col in jt_2.columns.values]
jt_2.reset_index(inplace=True)
jt_2.rename(columns={"season_":"season", "episode_":"episode"}, inplace=True)


jt_2.head(6)

,index,season,episode,total_mc_line_Chandler Bing,total_mc_line_Joey Tribbiani,total_mc_line_Monica Geller,total_mc_line_Phoebe Buffay,total_mc_line_Rachel Green,total_mc_line_Ross Geller,total_mc_scene_Chandler Bing,...,total_mc_scene_Monica Geller,total_mc_scene_Phoebe Buffay,total_mc_scene_Rachel Green,total_mc_scene_Ross Geller,total_mc_exclam_line_Chandler Bing,total_mc_exclam_line_Joey Tribbiani,total_mc_exclam_line_Monica Geller,total_mc_exclam_line_Phoebe Buffay,total_mc_exclam_line_Rachel Green,total_mc_exclam_line_Ross Geller
0,0,1,1,39.0,39.0,73.0,19.0,48.0,47.0,9.0,...,9.0,6.0,8.0,8.0,8.0,11.0,20.0,5.0,15.0,8.0
1,1,1,2,16.0,8.0,28.0,14.0,38.0,68.0,4.0,...,5.0,4.0,6.0,9.0,2.0,1.0,5.0,9.0,8.0,10.0
2,2,1,3,36.0,28.0,52.0,38.0,26.0,33.0,9.0,...,10.0,8.0,8.0,8.0,13.0,9.0,5.0,12.0,5.0,4.0
3,3,1,4,32.0,24.0,47.0,31.0,43.0,40.0,10.0,...,8.0,7.0,9.0,9.0,8.0,9.0,11.0,12.0,14.0,5.0
4,4,1,5,35.0,36.0,33.0,NaN,37.0,40.0,7.0,...,8.0,NaN,6.0,7.0,3.0,2.0,3.0,NaN,5.0,2.0
5,5,1,6,58.0,31.0,34.0,10.0,15.0,22.0,7.0,...,6.0,5.0,5.0,5.0,9.0,11.0,12.0,5.0,4.0,5.0


# Processing 3/4: Adding Episode Review/Viewership Info

In [205]:
#Taking Existing Episode df and dropping certain attributes
jt_3 = episodes.drop(columns=["title", "directed_by", "written_by", "air_date"])

jt_3.head(6)

,season,episode,us_views_millions,imdb_rating
0,1,1,21.5,8.3
1,1,2,20.2,8.1
2,1,3,19.5,8.2
3,1,4,19.7,8.1
4,1,5,18.6,8.5
5,1,6,18.2,8.1


# Processing 4/4: Joining Tables

In [206]:
#JOINING ALL STATISTICS INTO A SINGLE DATA TO CREATE A MODEL FROM

joined_table_final = pd.merge(pd.merge(jt_1, jt_2, on=["season", "episode"]), jt_3, on=["season", "episode"])

joined_table_final.head(6)

,season,episode,total_words,total_lines,total_scenes,total_speakers,total_oc_lines,total_oc_scenes,total_oc_exclam_lines,index,...,total_mc_scene_Rachel Green,total_mc_scene_Ross Geller,total_mc_exclam_line_Chandler Bing,total_mc_exclam_line_Joey Tribbiani,total_mc_exclam_line_Monica Geller,total_mc_exclam_line_Phoebe Buffay,total_mc_exclam_line_Rachel Green,total_mc_exclam_line_Ross Geller,us_views_millions,imdb_rating
0,1,1,292,16876,15,11,27,9,5,0,...,8.0,8.0,8.0,11.0,20.0,5.0,15.0,8.0,21.5,8.3
1,1,2,241,12249,11,14,69,8,19,1,...,6.0,9.0,2.0,1.0,5.0,9.0,8.0,10.0,20.2,8.1
2,1,3,237,12070,14,10,24,6,3,2,...,8.0,8.0,13.0,9.0,5.0,12.0,5.0,4.0,19.5,8.2
3,1,4,248,12532,16,14,31,6,10,3,...,9.0,9.0,8.0,9.0,11.0,12.0,14.0,5.0,19.7,8.1
4,1,5,238,12626,16,10,34,10,2,4,...,6.0,7.0,3.0,2.0,3.0,NaN,5.0,2.0,18.6,8.5
5,1,6,206,12700,9,8,36,4,2,5,...,5.0,5.0,9.0,11.0,12.0,5.0,4.0,5.0,18.2,8.1


# Inference Task

In [1]:
# I want to verify that lots of exclamation changes episode IMDb rating 

# Resampling import to use in bootstrapping 
from sklearn.utils import resample

# Two populations: IMDb ratings and Viewership 
df_regr = (master_list
    .groupby(['episode', 'season'])
    .agg({'Speaker_Exclaim': 'sum', 'Speaker_Exclaimed_Toward': 'sum', 'us_views_millions': 'mean', 'imdb_rating': 'mean'})
    .reset_index()
)
mean_exclamation = np.mean(df['Speaker_Exclaim'])
pop1 = df[df['Speaker_Exclaim']>mean_exclamation]['imdb_rating']
pop2 = df['imdb_rating']

# Define number of bootstrap resamples
n_resamples = 1000
mean_diffs = np.zeros(n_resamples)

#  Bootstrapping procedure (add difference in each resample mean to the list)
for i in range(n_resamples):
    s1 = resample(pop1, replace=True)
    s2 = resample(pop2, replace=True)
    mean_diffs[i] = np.mean(s1)-np.mean(s2) 
    
# Confidence interval 
ci_95 = np.percentile(mean_diffs, [2.5, 97.5]) 

# Mean Difference 
mean_difference = np.mean(mean_diffs)

